# imports

In [165]:
import sys
# point this to appropriate snudda installation 
sys.path.append('/Applications/Blender.app/Contents/Resources/4.0/python/lib/python3.10/site-packages/Snudda')

import bpy
import bmesh

import os
import glob
import h5py

import mathutils
import math
import random
import numpy as np

from snudda.utils.snudda_path import snudda_parse_path, get_snudda_data
from snudda import SnuddaLoad
snudda_data  = '/Users/wst/Desktop/Karolinska/Simulation/Neuron'

import visualisation_tools as vt

import importlib
importlib.reload(vt)

coord_conv = {'Mouselight': {'X':4, 'Y':3, 'Z':2}, 'Peng': {'X':2, 'Y':3, 'Z':4}}


### Setup blender space

In [170]:
vt.clear_scene()

### Draw a set of reconstructed neurons
By convention, these are centered at (0,0,0). We will create an array of morphologies and animate rotation about 'Z'.


In [171]:
morph_dir = '/Users/wst/Desktop/Karolinska/Simulation/Neuron/neurons/medial_og_gh'

lx = 0
ly = 0
offset = 400

for morph_path in glob.glob(os.path.join(morph_dir, '**', '*.swc')):
    name = morph_path.split('/')[-1].replace('.swc', '')
    vt.build_from_swc(filepath = morph_path, name = name, lx = lx, ly = ly, draw_axon = False, rotating = True)
    lx += offset
    if lx >= 4*offset:
        lx = 0
        ly += offset

vt.frame_all()


Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)


### Import meshes from Allen
Requires .obj mesh files for Allen CCF regions. Can be downloaded via allenSDK. 
Calculate region volume while you're at it. 

OBS: Allen meshes are bilateral by default.

In [153]:
meshdir = '/Users/wst/Desktop/Karolinska/Meshes/Allen Meshes/ccf'
regions = ['SNr', 'STN', 'GPe', 'CP', 'root']

colours = {'SNr': (52/255,168/255,224/255, 0.2),
           'STN': (232/255, 61/255, 61/255, 0.2), 
           'GPe': (161/255, 85/255, 168/255, 0.2), 
           'CP': (112/255, 182/255, 44/255, 0.2), 
           'root': (0, 0, 0, 0.01)}

for region in regions:
    obj = vt.import_allen_mesh(filepath = os.path.join(meshdir, f'{region}.obj'), colour = colours[region], alpha =colours[region][-1])
    print(f'{region} volume: {vt.get_mesh_volume(obj):.2f} mm^3')
    
vt.frame_selected()


OBJ import of 'SNr.obj' took 45.8 ms
SNr volume: 1.50 mm^3
OBJ import of 'STN.obj' took 4.5 ms
STN volume: 0.14 mm^3
OBJ import of 'GPe.obj' took 10.1 ms
GPe volume: 1.46 mm^3
OBJ import of 'CP.obj' took 35.3 ms
CP volume: 26.00 mm^3
OBJ import of 'root.obj' took 208.7 ms
root volume: 516.35 mm^3


### Draw a neuron from an external dataset
e.g. Peng, H., Xie, P., Liu, L. et al. Morphological diversity of single neurons in molecularly defined cell types. Nature 598, 174–181 (2021). https://doi.org/10.1038/s41586-021-03941-1

The morphologies in this dataset are not centered at (0,0,0). Therefore, location offsets can be ignored when drawing from the .swc file. 

OBS: drawing the axons for these neurons can be very slow, as the reconstructions are very detailed.

In [27]:
swc_path = '/Users/wst/Desktop/Karolinska/ReferenceData/CCFv3/182725_2241_x9113_y11193.swc'
vt.build_from_swc(filepath = swc_path, name = 'example', coord_space = coord_conv['Peng'], draw_axon = False)


Info: Removed 58 vertice(s)


### Draw somas from a snudda simulation

In [147]:
network_path = '/Users/wst/Desktop/Karolinska/Simulation/Neuron/networks/vis_test/'
sl = SnuddaLoad(os.path.join(network_path, 'network-synapses.hdf5'))
neuron_coords = sl.data["neuron_positions"]

coords_mesh = vt.draw_somas(neuron_coords, name = 'neuron coordinates', colour = (1,0,0.5,1), res = 2, scale_f = 1e6)
vt.flip_mesh_across_midline(coords_mesh) ## flip them to the other hemisphere, so we can also look at the full morphologies


### Draw neuron morphologies from a snudda network

In [136]:
network_path = '/Users/wst/Desktop/Karolinska/Simulation/Neuron/networks/vis_test/'
sl = SnuddaLoad(os.path.join(network_path, 'network-synapses.hdf5'))
neurons = sl.data["neurons"]

for neuron_id in range(20): #range(len(neurons)): 
    obj = vt.draw_neuron_from_snudda(neurons = neurons, neuron_id = neuron_id, snudda_data = snudda_data, name = f'neuron_{neuron_id}')


Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)


### Draw a neuron from a snudda network, with all of its postsynaptic partners

In [158]:
network_path = '/Users/wst/Desktop/Karolinska/Simulation/Neuron/networks/vis_test/'

post_ids, synapse_coords = vt.draw_presynaptic(network_path, pre_id = 0, snudda_data = snudda_data)


Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)


### Draw a neuron from a snudda network, with all of its presynaptic partners

In [154]:
network_path = '/Users/wst/Desktop/Karolinska/Simulation/Neuron/networks/vis_test/'

pre_ids, synapse_coords = vt.draw_postsynaptic(network_path, post_id = 23, snudda_data = snudda_data, match_synapses = True)


Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 2 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
